In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image
import numpy as np
import json
from geopy.distance import geodesic
import networkx as nx

In [19]:
import import_ipynb
import data_manip_weather as weather

In [20]:
class Df_and_graph:
    def __init__(self, edges_file = 'edges.json', fixes_file = 'fixes.json', weather_img = "weather.png" , weather_json = "weather.json"):
        self.df_combined = self.create_df_from_edges_and_fixes(edges_file, fixes_file)
        self.calculate_distance()
        
        self.weather = weather.Weather(weather_img, weather_json)
        
    def create_df_from_edges_and_fixes(self, edges_file, fixes_file):
        df_edges = pd.read_json(edges_file).rename(columns={0: 'A', 1: 'B'})
        df_fixes = pd.read_json('fixes.json')
        df_fixes = df_fixes.reindex(sorted(df_fixes.columns), axis=1).astype(np.float64).T
        
        df_combined = df_edges
        df_combined["A_latitude"] = df_combined['A'].map(df_fixes["latitude"])
        df_combined["A_longitude"] = df_combined['A'].map(df_fixes["longitude"])
        df_combined["B_latitude"] = df_combined['B'].map(df_fixes["latitude"])
        df_combined["B_longitude"] = df_combined['B'].map(df_fixes["longitude"])
        return df_combined
    def calculate_distance(self):
        self.df_combined["Distance"] = self.df_combined.apply(lambda x: geodesic([x["A_latitude"], x["A_longitude"]] , [x["B_latitude"], x["B_longitude"]]).km, axis=1)
    
    def weather_proportion_for_edges(self):
        self.df_combined["Weather"] = self.df_combined.apply(lambda x:  self.weather.weight_between_points([x["A_latitude"], x["A_longitude"]] , [x["B_latitude"], x["B_longitude"]]) , axis=1)
       
    def weather_weight_calculate(self, good = 0, moderate = 10, severe = 20):
        self.df_combined["Weather_weight"] = self.df_combined.apply(lambda x:   , axis=1)
        
    def weather_update(self,  weather_img, weather_json):
        self.weather = weather.Weather(weather_img, weather_json)
        self.weather_proportion_for_edges()
        

In [21]:
data = Df_and_graph()

In [22]:
data.df_combined

,A,B,A_latitude,A_longitude,B_latitude,B_longitude,distance
0,ABQ,ADDVL,35.043794,-106.816311,33.371758,-96.361231,980.724944
1,ABQ,AHAMU,35.043794,-106.816311,34.702697,-95.864267,1001.543351
2,ABQ,AKO,35.043794,-106.816311,40.155553,-103.179711,651.776859
3,ABQ,ALS,35.043794,-106.816311,37.349158,-105.815536,271.177389
4,ABQ,ANTHY,35.043794,-106.816311,31.310189,-100.543003,716.530016
...,...,...,...,...,...,...,...
116103,ZUVKU,WISEP,40.876139,-124.039036,42.250111,-123.354881,162.920422
116104,ZUVKU,WURUK,40.876139,-124.039036,44.412092,-120.921144,468.602018
116105,ZUVKU,YAPUB,40.876139,-124.039036,38.322275,-122.949075,298.595223
116106,ZUVKU,ZIDAX,40.876139,-124.039036,42.246989,-122.542739,196.868845


In [23]:
data.weather_proportion_for_edges()

ValueError: Columns must be same length as key

In [ ]:
data.df_combined

In [202]:
df_edges = pd.read_json('edges.json')
df_edges = df_edges.rename(columns={0: 'A', 1: 'B'})

df_fixes = pd.read_json('fixes.json')
df_fixes = df_fixes.reindex(sorted(df_fixes.columns), axis=1).astype(np.float64).T

In [203]:
df_fixes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1103 entries, ABQ to ZUVKU
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   latitude   1103 non-null   float64
 1   longitude  1103 non-null   float64
dtypes: float64(2)
memory usage: 25.9+ KB


In [ ]:
df_edges.info()

In [ ]:
w = weather.Weather()

In [ ]:
m = w.wmatrix()

In [ ]:
m

In [ ]:
df_edges[df_edges['A'] == "ADDVL"]

In [ ]:
df_fixes

In [ ]:
geodesic([df_fixes.loc["KSFO"][0], df_fixes.loc["KSFO"][1]] ,[ df_fixes.loc["KJFK"][0], df_fixes.loc["KJFK"][1] ]).km

In [ ]:
df_combined = df_edges

In [ ]:
df_combined["A_latitude"] = df_combined['A'].map(df_fixes["latitude"])
df_combined["A_longitude"] = df_combined['A'].map(df_fixes["longitude"])
df_combined["B_latitude"] = df_combined['B'].map(df_fixes["latitude"])
df_combined["B_longitude"] = df_combined['B'].map(df_fixes["longitude"])

In [ ]:
df_combined

In [ ]:
df_combined["distance"] = df_combined.apply(lambda x: geodesic([x["A_latitude"], x["A_longitude"]] , [x["B_latitude"], x["B_longitude"]]).km, axis=1)

In [ ]:
df_combined

In [ ]:
plt.hist(df_combined["distance"], 20)

In [ ]:
G = nx.Graph()
G.add_edge(1, 2)  # default edge data=1
G.add_edge(2, 3, weight=0.9)

In [ ]:
G

In [ ]:
nx.draw(G, with_labels = True)

In [ ]:
elist = [('a', 'b', 1.0), ('b', 'c', 3.0), ('a', 'c', 1.0), ('c', 'd', 7.3)]
G = nx.Graph()

G.add_weighted_edges_from(elist)

In [ ]:
G

In [ ]:
nx.draw(G, with_labels = True)

In [ ]:
nx.astar_path(G, 'a', 'd')

In [ ]:
G = nx.from_pandas_edgelist(df_combined, 'A', 'B', ['distance'])

In [ ]:
G

In [ ]:
pos=nx.spring_layout(G)

In [ ]:
nx.draw(G, pos)

In [ ]:
nx.astar_path(G, 'ABQ', 'ADDVL')

In [ ]:
nx.astar_path(G, 'ADDVL', 'ABQ')

In [ ]:
nx.astar_path(G, 'KSFO', 'KJFK')

In [ ]:
nx.path_weight(G, nx.astar_path(G, 'KSFO', 'KJFK'), weight = "distance")

In [ ]:
df_combined.describe()

In [ ]:
nx.dijkstra_path(G, 'KSFO', 'ZORUN')